# Ensemble methods

### Read the input files

In [128]:
import numpy as np
import os

XTrain = []
YTrain = []
XTest = []
YTest = []

#XTrain = np.loadtxt(os.getcwd() + r'/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt', delimiter=" ", dtype='str')
xTrainFileName = os.getcwd() + r'/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt'
with open(xTrainFileName, "r") as myfile:
    for line in myfile:
        XTrain.append(map(float, line.split()))
        
              
yTrainFileName = os.getcwd() + r'/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt'
with open(yTrainFileName, "r") as myfile:
    for line in myfile:
        YTrain.append(int(line.strip()))

XTrain = np.asarray(XTrain)
YTrain = np.asarray(YTrain)

xTestFileName = os.getcwd() + r'/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt'
with open(xTestFileName, "r") as myfile:
    for line in myfile:
        XTest.append(map(float, line.split()))
        
              
yTestFileName = os.getcwd() + r'/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt'
with open(yTestFileName, "r") as myfile:
    for line in myfile:
        YTest.append(int(line.strip()))
        
        
XTest = np.asarray(XTest)
YTest = np.asarray(YTest)

### Preprocess the time series data using Fast Fourier Transform

In [129]:
from scipy.fftpack import fft
# The first 265 columns represent the acceleration for all the axes in the Time Series Domain
xfTrain = fft(XTrain[:,0:265])
xfTest = fft(XTest[:,0:265])

# timeSeriesFFT = []
# for row in xf:
#     timeSeriesFFT.append(row)

# print (np.array(timeSeriesFFT)).shape

### Concatenate the acceleration for all the axes into one feature vector.

In [130]:
XTrain = np.append(XTrain[:,265:], xfTrain , 1)
XTest = np.append(XTest[:,265:], xfTest , 1)

### Try out different classifiers

In [131]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
def classifyRandomForestClassifier(XTrain, XTest, YTrain, YTest):
    clf = RandomForestClassifier(n_estimators=10)
    clf.fit(XTrain, YTrain)
    YPred = clf.predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size), YPred

In [132]:
#K Nearest Neighbours Classifier
from sklearn.neighbors import KNeighborsClassifier
def classifyKNNClassifier(XTrain, XTest, YTrain, YTest):
    neigh = KNeighborsClassifier(n_neighbors=3)
    YPred = neigh.fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size), YPred

In [133]:
#Gaussian Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
def classifyGaussianNaiveBayesClassifier(XTrain, XTest, YTrain, YTest):
    gnb = GaussianNB()
    YPred = gnb.fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size), YPred

In [134]:
#Multi Class SVM
from sklearn import svm
from sklearn.svm import NuSVC
def classifyMultiClassSVMClassifier(XTrain, XTest, YTrain, YTest):
    YPred = svm.SVC(kernel='linear').fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size), YPred

### Classify the signals into different activities. <br> </br> Use Leave-One-Out crossvalidation, where you test on one user and train on the rest of them.

In [135]:
from sklearn import cross_validation
X = xf
y = YTrain
N = 7352
loo = cross_validation.LeaveOneOut(N)
print loo
y = y.ravel()

# score = 0.0
# for train_index, test_index in loo:
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     score += classifyRandomForestClassifier(X_train, X_test,  y_train, y_test)

# score = score/N


sklearn.cross_validation.LeaveOneOut(n=7352)


In [136]:
#Stratified K Fold Cross Validation
from sklearn.cross_validation import StratifiedKFold
def stratifiedKFoldVal(XTrain, YTrain, classify):
    n_folds = 10
    totalScore = 0.0
    skf = StratifiedKFold(YTrain, n_folds)
    for train_index, test_index in skf:
        X_train, X_test = XTrain[train_index], XTrain[test_index]
        y_train, y_test = YTrain[train_index], YTrain[test_index]
        score, YPred = classify(X_train, X_test,  y_train, y_test)
        totalScore += score
    return totalScore/n_folds

In [137]:
classify = classifyRandomForestClassifier
score = stratifiedKFoldVal(XTrain, YTrain, classify)
print score

90.3063892113


In [138]:
classify = classifyKNNClassifier
score = stratifiedKFoldVal(XTrain, YTrain, classify)
print score

86.9656713006


In [139]:
classify = classifyGaussianNaiveBayesClassifier
score = stratifiedKFoldVal(XTrain, YTrain, classify)
print score

53.9947542962


In [140]:
classify = classifyMultiClassSVMClassifier
score = stratifiedKFoldVal(XTrain, YTrain, classify)
print score

93.6985423391


### Implement a Python framework for mixture of experts, 
### where the user can assign multiple classifiers to a different region of the feature space.

In [152]:
from sklearn.cross_validation import train_test_split
np.random.seed(9)
X_train, X_test, y_train, y_test = train_test_split(XTrain, YTrain, test_size=0.7)

In [174]:
def EnsembleFeatureSplitClassifiers(X_train, X_test, y_train, y_test, splitFeatIdx, classifiersList = [classifyRandomForestClassifier, classifyKNNClassifier, classifyGaussianNaiveBayesClassifier, classifyMultiClassSVMClassifier]):
    assert(len(classifiersList)*2,len(splitFeatIdx))
    
#     s1,YPred1 = classifyRandomForestClassifier(X_train[:,splitFeatIdx[0]:splitFeatIdx[1]], X_test[:,splitFeatIdx[0]:splitFeatIdx[1]], y_train, y_test)
#     s2,YPred2 = classifyKNNClassifier(X_train[:,splitFeatIdx[2]:splitFeatIdx[3]], X_test[:,splitFeatIdx[2]:splitFeatIdx[3]], y_train, y_test)
#     s3,YPred3 = classifyGaussianNaiveBayesClassifier(X_train[:,splitFeatIdx[4]:splitFeatIdx[5]], X_test[:,splitFeatIdx[4]:splitFeatIdx[5]], y_train, y_test)
#     s4,YPred4 = classifyMultiClassSVMClassifier(X_train[:,splitFeatIdx[6]:splitFeatIdx[7]], X_test[:,splitFeatIdx[6]:splitFeatIdx[7]], y_train, y_test)
#     # Combine Y Predicted values from different classifiers
#     XCombinedOp = np.column_stack((YPred1, YPred2, YPred3, YPred4))

    YPred = []
    i = 0
    for classifier in classifiersList:
        s,ypred = classifier(X_train[:,splitFeatIdx[i]:splitFeatIdx[i+1]], X_test[:,splitFeatIdx[i]:splitFeatIdx[i+1]], y_train, y_test)
        i = i + 2
        # Combine Y Predicted values from different classifiers
        YPred.append(ypred)
    
    XCombinedOp = np.column_stack(YPred)
    
    return XCombinedOp

<ipython-input-174-c0f6a9d7fa54>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(classifiersList)*2,len(splitFeatIdx))


### The results of these classifiers are forwarded to an additional classifier 
### (e.g. neural network) that is trained to optimally combine the output from the first stage.

In [175]:
from sknn.mlp import Classifier, Layer

def NeuralNetworkCombiner(XCombinedOpTrain, y_train, XCombinedOpTest, YTest):
    nn = Classifier(layers=[Layer("Rectifier", units=100),Layer("Linear")],
        learning_rate=0.02,
        n_iter=10)
    nn.fit(XCombinedOpTrain, y_train)
    YTestPred = nn.predict(XCombinedOpTest)
    diff = YTestPred - YTest.reshape(YTestPred.shape)
    score = diff[diff == 0].size
    score = (100.0 * score)/(YTestPred.size)
    return score

### Using disjoint features for all classifiers

In [177]:
featSplit = [0,265,265,420,420,555,555,561]

# With only 1 classifier
classifiersList = [classifyKNNClassifier]
XCombinedOpTrain = EnsembleFeatureSplitClassifiers(X_train, X_test, y_train, y_test, featSplit, classifiersList)
XCombinedOpTest = EnsembleFeatureSplitClassifiers(X_train, XTest, y_train, YTest, featSplit, classifiersList)
NeuralNetworkCombiner(XCombinedOpTrain, y_test, XCombinedOpTest, YTest)

58.46623685103495

In [178]:
# With 2 classifiers
classifiersList = [classifyKNNClassifier, classifyGaussianNaiveBayesClassifier]

XCombinedOpTrain = EnsembleFeatureSplitClassifiers(X_train, X_test, y_train, y_test, featSplit, classifiersList)
XCombinedOpTest = EnsembleFeatureSplitClassifiers(X_train, XTest, y_train, YTest, featSplit, classifiersList)
NeuralNetworkCombiner(XCombinedOpTrain, y_test, XCombinedOpTest, YTest)

63.48829317950458

In [181]:
# With 3 classifiers
classifiersList = [classifyKNNClassifier, classifyGaussianNaiveBayesClassifier, classifyMultiClassSVMClassifier]
featSplit = [0,265,265,420,420,555,555,561]

XCombinedOpTrain = EnsembleFeatureSplitClassifiers(X_train, X_test, y_train, y_test, featSplit, classifiersList)
XCombinedOpTest = EnsembleFeatureSplitClassifiers(X_train, XTest, y_train, YTest, featSplit, classifiersList)
NeuralNetworkCombiner(XCombinedOpTrain, y_test, XCombinedOpTest, YTest)

49.541907024092296

In [182]:
# With 4 classifiers
classifiersList = [classifyRandomForestClassifier, classifyKNNClassifier, classifyGaussianNaiveBayesClassifier, classifyMultiClassSVMClassifier]
featSplit = [0,265,265,420,420,555,555,561]

XCombinedOpTrain = EnsembleFeatureSplitClassifiers(X_train, X_test, y_train, y_test, featSplit)
XCombinedOpTest = EnsembleFeatureSplitClassifiers(X_train, XTest, y_train, YTest, featSplit)
NeuralNetworkCombiner(XCombinedOpTrain, y_test, XCombinedOpTest, YTest)

64.47234475738038

### Does the accuracy change if you use overlapping regions instead of disjunct ones?

In [172]:
# Using overlapping features for all classifiers
featSplit = [0,220,100,280,170,345,280,561]
XCombinedOpTrain = EnsembleFeatureSplitClassifiers(X_train, X_test, y_train, y_test, featSplit)
XCombinedOpTest = EnsembleFeatureSplitClassifiers(X_train, XTest, y_train, YTest, featSplit)
NeuralNetworkCombiner(XCombinedOpTrain, y_test, XCombinedOpTest, YTest)

72.31082456735663

In [170]:
# Using all features for all classifiers
featSplit = [0,561,0,561,0,561,0,561]
XCombinedOpTrain = EnsembleFeatureSplitClassifiers(X_train, X_test, y_train, y_test, featSplit)
XCombinedOpTest = EnsembleFeatureSplitClassifiers(X_train, XTest, y_train, YTest, featSplit)
NeuralNetworkCombiner(XCombinedOpTrain, y_test, XCombinedOpTest, YTest)

86.83406854428232

In [171]:
# import numpy as np
# from sklearn.neural_network import BernoulliRBM
# model = BernoulliRBM(n_components=2, n_iter=50)
# model.fit(XCombinedOpTrain, y_test)
# print model.intercept_hidden_
# print model.intercept_visible_
# print model.components_

### Graph of change of accuracy w.r.t. the number of used classifiers in the first stage.

### Is there any benefit of using mixture of experts on this dataset?